# Installing/importing Libraries

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 64.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 101.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 27.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import ast
from transformers import AutoTokenizer, TFBertForSequenceClassification
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, confusion_matrix, f1_score, classification_report

# Converting Class labels to Numerical Value

In [ ]:
def extractList(df):
    df.loc[df["Post"].str.endswith("]") == False, "Post"] = df.loc[df["Post"].str.endswith("]") == False, "Post"] + "']"
    df["Post"] = df["Post"].apply(lambda x: ast.literal_eval(x))
    df["Post"] = df["Post"].apply(lambda x: " ".join(x))

In [ ]:
label_conversion = {"Supportive": 0,
                    "Indicator": 1,
                    "Ideation": 2,
                    "Behavior": 3,
                    "Attempt": 4}

# Downloading Data From GitHub and Loading

In [ ]:
!git clone https://github.com/hrmoradi/Workshop_data # goes to your cotent folder

Cloning into 'Workshop_data'...
remote: Enumerating objects: 5865, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 5865 (delta 3), reused 23 (delta 2), pack-reused 5840
Receiving objects: 100% (5865/5865), 1.13 GiB | 31.96 MiB/s, done.
Resolving deltas: 100% (5/5), done.
Updating files: 100% (6261/6261), done.


In [ ]:
dat = pd.read_csv("/content/Workshop_data/500Reddit.txt")
dat

,User,Post,Label
0,user-0,"['Its not a viable option, and youll be leavin...",Supportive
1,user-1,['It can be hard to appreciate the notion that...,Ideation
2,user-2,"['Hi, so last night i was sitting on the ledge...",Behavior
3,user-3,['I tried to kill my self once and failed badl...,Attempt
4,user-4,['Hi NEM3030. What sorts of things do you enjo...,Ideation
...,...,...,...
495,user-495,"['Its not the end, it just feels that way. Or ...",Supportive
496,user-496,"['It was a skype call, but she ended it and Ve...",Indicator
497,user-497,['That sounds really weird.Maybe you were Dist...,Supportive
498,user-498,['Dont know there as dumb as it sounds I feel ...,Attempt


In [ ]:
# Rename Labels to integers in order of intensity
dat = dat.replace({"Label":label_conversion})
# Extract list of posts from string of list and concatenate together
extractList(dat)
dat

,User,Post,Label
0,user-0,"Its not a viable option, and youll be leaving ...",0
1,user-1,It can be hard to appreciate the notion that y...,2
2,user-2,"Hi, so last night i was sitting on the ledge o...",3
3,user-3,I tried to kill my self once and failed badly ...,4
4,user-4,Hi NEM3030. What sorts of things do you enjoy ...,2
...,...,...,...
495,user-495,"Its not the end, it just feels that way. Or at...",0
496,user-496,"It was a skype call, but she ended it and Vent...",1
497,user-497,That sounds really weird.Maybe you were Distra...,0
498,user-498,Dont know there as dumb as it sounds I feel Hy...,4


In [ ]:
num_labels = dat["Label"].nunique()
num_labels

5

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(dat["Post"], dat["Label"], test_size=0.33, random_state=42, stratify=dat["Label"], shuffle=True)

# Text To verctor Representation

In [ ]:
# Import BERT tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
X_train_input = tokenizer(X_train.values.tolist(), max_length = 512, truncation="longest_first", padding="max_length")
X_test_input = tokenizer(X_test.values.tolist(), max_length = 512, truncation="longest_first", padding="max_length")

In [ ]:
train_input_ids = np.asarray(X_train_input["input_ids"])
train_att_mask = np.asarray(X_train_input["attention_mask"])

test_input_ids = np.asarray(X_test_input["input_ids"])
test_att_mask = np.asarray(X_test_input["attention_mask"])

y_train = np.asarray(y_train)
y_test = np.asarray(y_test)

# Freezing transformer weights

In [ ]:
model_frozen = TFBertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels = num_labels)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model_frozen.bert.trainable = False
model_frozen.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  3845      
                                                                 
Total params: 109,486,085
Trainable params: 3,845
Non-trainable params: 109,482,240
_________________________________________________________________


In [ ]:
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = tf.keras.metrics.SparseCategoricalAccuracy(name='accuracy')

model_frozen.compile(optimizer=Adam(learning_rate=1e-5), 
                      loss=loss,
                      metrics=metrics)

In [ ]:
model_frozen.fit(x=[train_input_ids, train_att_mask], y=y_train, epochs=15, batch_size=4)

Epoch 1/15
84/84 [==============================] - 35s 196ms/step - loss: 1.5454 - accuracy: 0.3284
Epoch 2/15
84/84 [==============================] - 17s 198ms/step - loss: 1.5324 - accuracy: 0.3373
Epoch 3/15
84/84 [==============================] - 17s 202ms/step - loss: 1.5491 - accuracy: 0.3373
Epoch 4/15
84/84 [==============================] - 17s 198ms/step - loss: 1.5282 - accuracy: 0.3403
Epoch 5/15
84/84 [==============================] - 16s 195ms/step - loss: 1.5213 - accuracy: 0.3493
Epoch 6/15
84/84 [==============================] - 16s 196ms/step - loss: 1.5178 - accuracy: 0.3403
Epoch 7/15
84/84 [==============================] - 16s 196ms/step - loss: 1.5121 - accuracy: 0.3463
Epoch 8/15
84/84 [==============================] - 17s 199ms/step - loss: 1.5283 - accuracy: 0.3373
Epoch 9/15
84/84 [==============================] - 17s 198ms/step - loss: 1.5218 - accuracy: 0.3463
Epoch 10/15
84/84 [==============================] - 16s 196ms/step - loss: 1.5231 - accura

Since TFBertForSequenceClassification returns logits, we must first convert it to probabilities using softmax.

In [ ]:
output = model_frozen.predict([test_input_ids, test_att_mask])
softmax = tf.keras.layers.Softmax(axis=-1)
preds = softmax(output.logits)
pred_labels = preds.numpy().argmax(axis=1)

6/6 [==============================] - 9s 1s/step


In [ ]:
print('Classification Report')
print(classification_report(y_test,pred_labels))

Classification Report
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        36
           1       0.00      0.00      0.00        33
           2       0.34      1.00      0.51        56
           3       0.00      0.00      0.00        25
           4       0.00      0.00      0.00        15

    accuracy                           0.34       165
   macro avg       0.07      0.20      0.10       165
weighted avg       0.12      0.34      0.17       165



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
roc_auc_score(y_test, preds, multi_class='ovr', average='macro')

0.5649207643355789

# Unfreeze transformer weights

In [ ]:
model_unfrozen = TFBertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels = num_labels)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model_unfrozen.bert.trainable = True # Default value set to true
model_unfrozen.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_75 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  3845      
                                                                 
Total params: 109,486,085
Trainable params: 109,486,085
Non-trainable params: 0
_________________________________________________________________


In [ ]:
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = tf.keras.metrics.SparseCategoricalAccuracy(name='accuracy')

model_unfrozen.compile(optimizer=Adam(learning_rate=1e-5), 
                      loss=loss,
                      metrics=metrics)

In [ ]:
model_unfrozen.fit(x=[train_input_ids, train_att_mask], y=y_train, epochs=10, batch_size=4)

Epoch 1/10
84/84 [==============================] - 89s 531ms/step - loss: 1.5330 - accuracy: 0.3343
Epoch 2/10
84/84 [==============================] - 44s 519ms/step - loss: 1.4675 - accuracy: 0.4030
Epoch 3/10
84/84 [==============================] - 42s 505ms/step - loss: 1.3528 - accuracy: 0.4418
Epoch 4/10
84/84 [==============================] - 42s 501ms/step - loss: 1.2046 - accuracy: 0.5075
Epoch 5/10
84/84 [==============================] - 42s 504ms/step - loss: 1.0752 - accuracy: 0.5612
Epoch 6/10
84/84 [==============================] - 43s 508ms/step - loss: 0.9360 - accuracy: 0.6239
Epoch 7/10
84/84 [==============================] - 42s 498ms/step - loss: 0.6968 - accuracy: 0.7403
Epoch 8/10
84/84 [==============================] - 42s 499ms/step - loss: 0.4921 - accuracy: 0.8507
Epoch 9/10
84/84 [==============================] - 42s 497ms/step - loss: 0.3116 - accuracy: 0.9582
Epoch 10/10
84/84 [==============================] - 42s 498ms/step - loss: 0.2081 - accura

Since TFBertForSequenceClassification returns logits, we must first convert it to probabilities using softmax.

In [ ]:
output = model_unfrozen.predict([test_input_ids, test_att_mask])
softmax = tf.keras.layers.Softmax(axis=-1)
preds = softmax(output.logits)
pred_labels = preds.numpy().argmax(axis=1)

6/6 [==============================] - 11s 1s/step


In [ ]:
print('Classification Report')
print(classification_report(y_test,pred_labels))

Classification Report
              precision    recall  f1-score   support

           0       0.62      0.50      0.55        36
           1       0.29      0.33      0.31        33
           2       0.43      0.62      0.51        56
           3       0.53      0.32      0.40        25
           4       1.00      0.13      0.24        15

    accuracy                           0.45       165
   macro avg       0.58      0.38      0.40       165
weighted avg       0.51      0.45      0.44       165



In [ ]:
roc_auc_score(y_test, preds, multi_class='ovr', average='macro')

0.6940348031467958

# Create your own traditional model

In [ ]:
import tensorflow as tf
import pandas as pd

# # uncomment for IMDB dataset # 3 lines below
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.imdb.load_data(num_words=10000)
x_train = tf.keras.preprocessing.sequence.pad_sequences(X_train, maxlen=512)
x_test = tf.keras.preprocessing.sequence.pad_sequences(X_test, maxlen=512)

# Input for variable-length sequences of integers
inputs = tf.keras.Input(shape=(None,), dtype="int32")

# # uncomment vectorization for suicidal datae set
# vectorize_layer = tf.keras.layers.TextVectorization(
#     standardize='lower_and_strip_punctuation',
#     split='whitespace',
#     ngrams=None,
#     max_tokens=10000,
#     output_mode='int',
#     output_sequence_length=512,
#     pad_to_max_tokens=True)
# vectorize_layer.adapt(X_train)
# x_train = vectorize_layer(X_train) 
# x_test = vectorize_layer(X_test) 

# Embed each integer in a 100-dimensional vector
x = tf.keras.layers.Embedding(10000, 100)(inputs) # top 10k

# Add 2 bidirectional LSTMs
x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True))(x)
x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64))(x)
# Add a classifier
outputs = tf.keras.layers.Dense(1, activation="sigmoid")(x)
model = tf.keras.Model(inputs, outputs)

model.compile("adam", "binary_crossentropy", metrics=["accuracy"])
model.fit(x_train, y_train, batch_size=64, epochs=1) # , validation_data=(x_test, y_test)

print(model.metrics_names)
model.evaluate(x_test, y_test)

391/391 [==============================] - 74s 167ms/step - loss: 0.4119 - accuracy: 0.8106
['loss', 'accuracy']
782/782 [==============================] - 26s 31ms/step - loss: 0.3287 - accuracy: 0.8665


[0.3287408947944641, 0.8664799928665161]